In [ ]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_datareader.data as web
from datetime import datetime, date
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.preprocessing import minmax_scale
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, LSTM,Dropout
import keras
import h5py as h5py
import cython
from sklearn.metrics import mean_squared_error, mean_absolute_error
from matplotlib import pyplot

In [ ]:
import os
os.getcwd()
os.chdir('D:\Stanford\CS230\Project\Final')
os.getcwd()

In [ ]:
stock_data = pd.read_csv('historical_stock_prices.csv')
stock_data['date'] = pd.to_datetime(stock_data.date)
start_date = '2003-01-01'
end_date = '2013-12-31'
stock_data0313 = stock_data.loc[(stock_data['date'] > start_date) & (stock_data['date'] <= end_date)]

In [ ]:
key_stats = pd.read_csv('key_stats_yahoo.csv')
key_stats.Ticker = key_stats['Ticker'].str.upper()
key_stats.Date = pd.to_datetime(key_stats.Date)
key_stats.Date = key_stats.Date.map(lambda x: x.strftime('%Y-%m-%d'))
key_stats.Date = pd.to_datetime(key_stats.Date)
key_stats = key_stats.rename(columns = {'Date':'date', 'Ticker':'ticker'})

In [ ]:
#group by market cap, and output the first 300 stocks
key_stats_grouped = key_stats.groupby(by='ticker').sum()
market_cap_ranked = key_stats_grouped['Market Cap']
market_cap_ranked['BAC'] = 0
market_cap_ranked = market_cap_ranked.sort_values(ascending=False)
market_cap_ranked_names = market_cap_ranked.to_frame()
market_cap_ranked_names = market_cap_ranked_names.reset_index()['ticker']

num_stocks = 10

top_stocks = market_cap_ranked_names[:num_stocks]
top_stocks


In [ ]:
#Merge the key statistic into the prices
Price_stats0313 = pd.merge(stock_data0313, key_stats, how = 'left', on=['ticker','date'])

In [ ]:
return_template = Price_stats0313[Price_stats0313.ticker == 'GOOG']
return_template = return_template.loc[(return_template['date'] >= '2013-01-01') & (return_template['date'] <= '2013-12-31')]
return_template = return_template[['date']]
return_template = return_template.reset_index()
return_template_predicted = return_template.drop(columns=['index'])
lo_return = return_template_predicted
ls_return = return_template_predicted

In [ ]:
#number of days to use for the prediction
window_size = 20

train_loss_trend = {}
dev_loss_trend = {}
train_loss = {}
dev_loss = {}
test_mse = {}
hit_ratio_list={}
test_mae={}

In [ ]:
def create_rnn_data_structure(data, window_size):
    y = data[window_size:]['adj_close']
    n = data.shape[0]
    X = np.stack([data[i: j] for i, j in enumerate(range(window_size, n))], axis=0)
    return X, y

In [ ]:
for i in range(len(top_stocks)):

    #globals()[top_stocks[0] + '_data'] = Price_stats0313[Price_stats0313.ticker == top_stocks[0]]
    selected_data = Price_stats0313[Price_stats0313.ticker == top_stocks[i]]
    selected_data = selected_data.sort_values('date')

    #use forward fill to deal with missing data
    selected_data = selected_data.ffill(axis = 0)

    selected_data = selected_data.sort_values('date', ascending = False)
    selected_data = selected_data.ffill(axis = 0) 

    selected_data = selected_data.sort_values('date')

    selected_data_truncated = selected_data[selected_data['date'] > '2004-01-01']

    selected_data_truncated = selected_data_truncated.drop(columns=['Price','Unnamed: 0','ticker'])
    '''

    Model 1 Does not work
    selected_data_truncated = selected_data_truncated[['adj_close','date','volume','DE Ratio','Return on Equity',
                                                                'Price/Book','Profit Margin','Diluted EPS','Beta','Trailing P/E','Price/Sales'
                                                      ,'Return on Equity','PEG Ratio','Earnings Growth','Total Cash Per Share',
                                                      'Current Ratio','Book Value Per Share','Enterprise Value/EBITDA']]

    '''

    '''
    #Model 4 Works better
    selected_data_truncated = selected_data_truncated[['adj_close','date','volume','DE Ratio','Return on Equity',
                                                                'Price/Book','Profit Margin','Diluted EPS']]

    '''


    #Model 2  
    selected_data_truncated = selected_data_truncated[['adj_close','date','volume','DE Ratio','Return on Equity',
                                                                'Price/Book','Profit Margin','Diluted EPS','Beta']]

    '''
    #Model 3
    selected_data_truncated = selected_data_truncated[['adj_close','date','volume','DE Ratio','Return on Equity',
                                                                'Price/Book','Profit Margin','Diluted EPS','Beta',
                                                      'Return on Equity','PEG Ratio','Earnings Growth']]
    '''


    #selected_data_truncated.info()


    selected_data_truncated.set_index('date', inplace=True)
    #Apply min_max Scale
    selected_data_truncated_scaled = selected_data_truncated.apply(minmax_scale)


    #transform the data to fit the rnn
    X, y = create_rnn_data_structure(selected_data_truncated_scaled, window_size=window_size)


    #split the data into train,dev and test
    selected_data_truncated_scaled_reset_index = selected_data_truncated_scaled.reset_index()

    #We use 2012 data for development
    dev_size = selected_data_truncated_scaled_reset_index[(selected_data_truncated_scaled_reset_index['date'] > '2012-01-01') & (selected_data_truncated_scaled_reset_index['date'] <= '2012-12-31')].date.nunique()

    #we use 2013 data for testing
    test_size = selected_data_truncated_scaled_reset_index[(selected_data_truncated_scaled_reset_index['date'] > '2013-01-01') & (selected_data_truncated_scaled_reset_index['date'] <= '2013-12-31')].date.nunique()

    train_size = X.shape[0] - dev_size - test_size

    X_train, y_train = X[:train_size], y[:train_size]
    X_dev, y_dev = X[train_size:train_size + dev_size], y[train_size:train_size + dev_size]
    X_test, y_test = X[train_size + dev_size:train_size + dev_size + test_size], y[train_size + dev_size:train_size + dev_size+test_size]

    #X_train.shape, y_train.shape, X_dev.shape, y_dev.shape, X_test.shape, y_test.shape


    # design network as 2 LSTM with correposning dropout layer, together with 1 dense layer and one output layer
    model = Sequential()
    model.add(LSTM(50, activation='tanh', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]), name = 'LSTM1'))
    model.add(Dropout(0.2))
    model.add(LSTM(30, activation='tanh',name = 'LSTM2'))
    model.add(Dropout(0.2))
    model.add(Dense(10, name = 'FC1',activation='relu'))
    model.add(Dense(1, name = 'Output',activation='sigmoid'))
    model.compile(loss='mean_squared_error', optimizer='adam')


    # fit network
    stock_Model = model.fit(X_train, y_train, epochs=100, batch_size=30, validation_data=(X_dev, y_dev), verbose=2, shuffle=False)


    #Store the training and development total losses
    train_loss_trend[str(top_stocks[i])] = stock_Model.history['loss']
    dev_loss_trend[str(top_stocks[i])] = stock_Model.history['val_loss']
    train_loss[str(top_stocks[i])] = stock_Model.history['loss'][-1]
    dev_loss[str(top_stocks[i])] = stock_Model.history['val_loss'][-1]

    #Predict the future Price
    y_pred = pd.DataFrame(model.predict(X_test), index=y_test.index)

    #compute the mean squred error and mean squred error of the prediction
    test_mse_stats = mean_squared_error(y_pred, y_test)

    #Store the mean squred error for the testing stage
    test_mse[str(top_stocks[i])] = test_mse_stats
    
    
    #compute the mean absolute error and mean squred error of the prediction
    test_mae_stats = mean_absolute_error(y_pred, y_test)
    test_mae[str(top_stocks[i])] = test_mae_stats
    

    #Use the predicted value for bulding up the trading strategy
    daily_predicted_return = y_pred/y_pred.shift(1)-1



    #Compute the hit ratio
    true_hit = ((y_test/y_test.shift(1)-1)>0)*1
    predicted_hit = (daily_predicted_return > 0)*1
    true_array = np.asarray(true_hit)
    predicted_array = np.asarray(predicted_hit)
    true_array = np.reshape(true_array, (len(true_array), 1))
    hit_ratio = (true_array==predicted_array).sum()/len(predicted_hit)

    #Store the hit ratio
    hit_ratio_list[str(top_stocks[i])] = hit_ratio





    #Obtain the trading signal for long only strategy
    signal = (daily_predicted_return > 0)*1
    signal = signal.rename(columns = {0:'lo'})

    #Obtain the trading signal for long/short strategy
    variable_name = {1 : 1 , 0 : -1 }
    signal['ls'] = signal['lo'].map(variable_name)


    #extract the actual stock prices
    actual_prices = stock_data0313[stock_data0313.ticker == top_stocks[i]]

    #Get the first date and last date in the signal to cut the actual price in the correct range
    first_day_signal = signal.index[0]
    last_day_signal = signal.index[-1]
    actual_prices13 = actual_prices.loc[(actual_prices['date'] >= first_day_signal) & (actual_prices['date'] <= last_day_signal)]

    #compute the daily actual return, assuming we only trade intraday
    actual_prices13['daily_actual_return'] = actual_prices13.close/actual_prices13.open - 1


    #merge the signal and the return for computing the daily trading P&L
    signal_merge = signal.reset_index()
    signal_actual_return = pd.merge(signal_merge, actual_prices13, how='left', on='date')


    #Compute the long only return
    signal_actual_return['daily_lo_return_'+str(top_stocks[i])] = signal_actual_return.lo*signal_actual_return.daily_actual_return
    #Compute the long/short return
    signal_actual_return['daily_ls_return_'+str(top_stocks[i])] = signal_actual_return.ls*signal_actual_return.daily_actual_return
    #Keep only the useful columns
    signal_actual_return_truncated = signal_actual_return[['date', 'daily_lo_return_'+str(top_stocks[i]), 'daily_ls_return_'+str(top_stocks[i])]]

    #Merge the only only return into our dataset
    lo_return = pd.merge(lo_return, signal_actual_return_truncated, how = 'left', on='date')
    lo_return = lo_return.drop(columns='daily_ls_return_'+str(top_stocks[i]))


    #merge the short only return into our dataset
    ls_return = pd.merge(ls_return, signal_actual_return_truncated, how = 'left', on='date')
    ls_return = ls_return.drop(columns='daily_lo_return_'+str(top_stocks[i]))



In [ ]:
#Compute the portfolio Return
lo_return.set_index('date', inplace=True)
ls_return.set_index('date', inplace=True)
daily_lo_portfolio_return = lo_return.sum(axis=1)
daily_ls_portfolio_return = ls_return.sum(axis=1)

#Compute the portfolio Cummulative Return
lo_cum_return = (1 + daily_lo_portfolio_return).cumprod()
ls_cum_return = (1 + daily_ls_portfolio_return).cumprod()

In [ ]:
#output mse to csv
(pd.DataFrame.from_dict(data=test_mse, orient='index')
   .to_csv('model3_mse.csv', header=False))

In [ ]:

#Save the return data
lo_return.to_excel('D:\Stanford\CS230\Project\output\loLSTreturnM3.xlsx', index = None, header=True)
ls_return.to_excel('D:\Stanford\CS230\Project\output\lsLSTreturnM3.xlsx', index = None, header=True)


In [ ]:
#Save hit ratio
(pd.DataFrame.from_dict(data=hit_ratio_list, orient='index')
   .to_csv('model3_hitratio.csv', header=False))

In [ ]:
#Save mae
(pd.DataFrame.from_dict(data=test_mae, orient='index')
   .to_csv('model3_mae.csv', header=False))